In [1]:
from itertools import combinations

import numpy as np
import pandas as pd
from anytree import Node, RenderTree, findall

from AKB_distance import AKB_distance

## Build a Tree

In [2]:
# set tree structure
A = Node('A')
B = Node('B', parent=A)
C = Node('C', parent=A)
D = Node('D', parent=B)
E = Node('E', parent=B)
F = Node('F', parent=C)
G = Node('G', parent=C)
H = Node('H', parent=D)
I = Node('I', parent=E)
J = Node('J', parent=E)
K = Node('K', parent=F)
L = Node('L', parent=F)
M = Node('M', parent=G)
l1s = [B, C]
l2s = [D, E, F, G]
l3s = [H,  I, J, K, L, M]
for pre, fill, node in RenderTree(A):
    print("%s%s" % (pre, node.name))
# from anytree.exporter import DotExporter
# for line in DotExporter(true_tag):
#     print(line)

A
├── B
│   ├── D
│   │   └── H
│   └── E
│       ├── I
│       └── J
└── C
    ├── F
    │   ├── K
    │   └── L
    └── G
        └── M


In [3]:
l1_nodes = findall(A, filter_=lambda node: node.depth == 1)
l2_nodes = findall(A, filter_=lambda node: node.depth == 2)
l3_nodes = findall(A, filter_=lambda node: node.depth == 3)
nodes = tuple([A]) + l1_nodes + l2_nodes + l3_nodes

## Calculate AKB distance for each pair of nodes

In [4]:
dist_mat = pd.DataFrame(index=[node.name for node in nodes], columns=[node.name for node in nodes])
for node1 in nodes:
    for node2 in nodes:
        dist_mat.loc[node1.name, node2.name] = AKB_distance(node1, node2)

In [5]:
dist_mat

,A,B,C,D,E,F,G,H,I,J,K,L,M
A,0,0.768622,0.768622,1.172503,1.172503,1.172503,1.172503,1.576384,1.384727,1.384727,1.384727,1.384727,1.576384
B,0.768622,0,1.537244,0.403881,0.403881,1.941125,1.941125,0.807762,0.616105,0.616105,2.153348,2.153348,2.345005
C,0.768622,1.537244,0,1.941125,1.941125,0.403881,0.403881,2.345005,2.153348,2.153348,0.616105,0.616105,0.807762
D,1.172503,0.403881,1.941125,0,0.807762,2.345005,2.345005,0.403881,1.019986,1.019986,2.557229,2.557229,2.748886
E,1.172503,0.403881,1.941125,0.807762,0,2.345005,2.345005,1.211643,0.212224,0.212224,2.557229,2.557229,2.748886
F,1.172503,1.941125,0.403881,2.345005,2.345005,0,0.807762,2.748886,2.557229,2.557229,0.212224,0.212224,1.211643
G,1.172503,1.941125,0.403881,2.345005,2.345005,0.807762,0,2.748886,2.557229,2.557229,1.019986,1.019986,0.403881
H,1.576384,0.807762,2.345005,0.403881,1.211643,2.748886,2.748886,0,1.423867,1.423867,2.96111,2.96111,3.152767
I,1.384727,0.616105,2.153348,1.019986,0.212224,2.557229,2.557229,1.423867,0,0.424448,2.769453,2.769453,2.96111
J,1.384727,0.616105,2.153348,1.019986,0.212224,2.557229,2.557229,1.423867,0.424448,0,2.769453,2.769453,2.96111


## Verify some properties

In [6]:
# symmetry
dist_mat == dist_mat.T

,A,B,C,D,E,F,G,H,I,J,K,L,M
A,True,True,True,True,True,True,True,True,True,True,True,True,True
B,True,True,True,True,True,True,True,True,True,True,True,True,True
C,True,True,True,True,True,True,True,True,True,True,True,True,True
D,True,True,True,True,True,True,True,True,True,True,True,True,True
E,True,True,True,True,True,True,True,True,True,True,True,True,True
F,True,True,True,True,True,True,True,True,True,True,True,True,True
G,True,True,True,True,True,True,True,True,True,True,True,True,True
H,True,True,True,True,True,True,True,True,True,True,True,True,True
I,True,True,True,True,True,True,True,True,True,True,True,True,True
J,True,True,True,True,True,True,True,True,True,True,True,True,True


In [7]:
# triangle inequality
triangle_inequalities = []
for node1, node2, node3 in combinations(nodes, 3):
    triangle_inequality = dist_mat.loc[node1.name, node3.name] <= dist_mat.loc[node1.name, node2.name] + dist_mat.loc[node2.name, node3.name]
    triangle_inequalities.append(triangle_inequality)
all(triangle_inequalities)

True

In [8]:
# subpath additivity
dist_mat.loc[D.name, F.name] == dist_mat.loc[D.name, B.name] + dist_mat.loc[B.name, A.name] + dist_mat.loc[A.name, C.name] + dist_mat.loc[C.name, F.name]

True

In [9]:
# common ancestor
dist_mat.loc[D.name, C.name], dist_mat.loc[D.name, E.name]

(1.9411245210313408, 0.8077618947017187)